# Extract UNWTO Tourist estimates and use them to create adjustment factors for Alcohol LPC
### By Max Griswold

### 1. Set up global variables and file locations

In [8]:
import pandas as pd
from pandas import DataFrame
import string
import numpy as np
import statsmodels.api as sm
import os
from multiprocessing import Pool

#Starting datasets
raw = {}
proportion_datasets = {}
filtered_by_cp = {}

years = list(range(1980,1995,1) + [2015,2017])

##File locations
incoming = '/home/j/DATA/UNWTO_COMPENDIUM_TOURISM_STATISTICS/1995_2014'
outgoing = "/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/outputs"

template = pd.read_stata('/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/inputs/alc_template.dta')
template.rename(columns={'iso3':'ihme_loc_id'}, inplace=True)
template.drop_duplicates(['ihme_loc_id', 'year_id'])

#Variables used to handle draws later from alc lpc.
draws = ['draw_{}'.format(x) for x in range(1000)]
draws_old = ['draw_{}_old'.format(x) for x in range(1000)]

rename = dict(zip(draws, draws_old))

### 2. Read files from incoming folder, set name of dataframe to specific countries and store in dictionary.


### 3. Read specific sheets from raw datasets and store in filtered dictionaries

In [85]:
for files in os.listdir(incoming):
    if files.lower().endswith('.xlsx'):
        name = files.split('UNWTO_COMPENDIUM_TOURISM_STATISTICS_1995_2014_')
        name = name[1].split('_Y2016M01D12.XLSX')
        name = name[0].replace("_", " ")
        name = name.lower()
        name = string.capwords(name)
        data = pd.read_excel(incoming + '/' + files, 
                             sheetname=None, header=3, skiprows=2, na_values= ['..', '', 0], keep_default_na = False)
        raw[name] = data

#Make function to fix UNWTO location names not conforming to GBD location names
def fix_locations(data, more=False):
    location_fix = {'Antigua And Barbuda':'Antigua and Barbuda', 'Bahamas':'The Bahamas', 
                    'Bolivia, Plurinational State Of': 'Bolivia', 'Bosnia And Herzegovina':'Bosnia and Herzegovina', 
                    'Brunei Darussalam':'Brunei', 'Congo, Democratic Republic Of The':'Democratic Republic of the Congo', 
                    'Cote D Ivoire':"Cote d'Ivoire", 'Gambia':'The Gambia', 'Guinea-bissau':'Guinea-Bissau', 
                    'Hong Kong, China':'Hong Kong Special Administrative Region of China', 'Iran, Islamic Republic Of':'Iran', 
                    'Korea, Republic Of':'South Korea', 'Macao China':'Macao Special Administrative Region of China', 
                    'Micronesia, Federated States Of':'Federated States of Micronesia', 'Moldova, Republic of':'Moldova',
                    'Russian Federation':'Russia', 'Serbia and Montenegro':'Serbia',
                    'Saint Vincent And The Grenadines':'Saint Vincent and the Grenadines', 
                    'Sao Tome And Principe':'Sao Tome and Principe', 'State Of Palestine':'Palestine', 
                    'Syrian Arab Republic':'Syria', 'Tanzania, United Republic Of':'Tanzania', 'Timor Leste':'Timor-Leste', 
                    'Trinidad And Tobago':'Trinidad and Tobago', 'United States Of America':'United States', 
                    'United States Virgin Islands':'Virgin Islands, U.S.', 'Venezuela, Bolivarian Republic Of':'Venezuela', 
                    'Viet Nam':'Vietnam', 'The Former Yugoslav Republic of Macedonia':'Macedonia'}
    
    visitors_fix = {'Bahamas':'The Bahamas', 'Belgium / Luxembourg':'Belgium', 'Bolivia, Plurinational State of': 'Bolivia', 
                    'Brunei Darussalam':'Brunei', 'China + Hong Kong, China':'China', 
                    'Congo, Democratic Republic of the':'Democratic Republic of the Congo', "Côte d'Ivoire":"Cote d'Ivoire", 
                    'Czech Republic/Slovakia':'Czech Republic', 'Gambia':'The Gambia', 
                    'Hong Kong, China':'Hong Kong Special Administrative Region of China', 'India, Pakistan':'India', 
                    'Iran, Islamic Republic of':'Iran', 'Korea, Republic of':'South Korea', 'Serbia and Montenegro':'Serbia',
                    'Saint Vincent And The Grenadines':'Saint Vincent and the Grenadine
                    "Korea, Democratic People's Republic of":'South Korea', "Lao People's Democratic Republic":'Laos', 
                    'Macao, China':'Macao Special Administrative Region of China', 'Moldova, Republic of':'Moldova',
                    'Micronesia, Federated States of':'Federated States of Micronesia', 'Russian Federation':'Russia', 
                    'Spain,Portugal':'Spain', 'State of Palestine':'Palestine', 'Syrian Arab Republic':'Syria', 
                    'United Kingdom/Ireland':'United Kingdom', 'Taiwan Province of China':'Taiwan', 
                    'Tanzania, United Republic of':'Tanzania', 'United States of America':'United States', 
                    'United States Virgin Islands':'Virgin Islands, U.S.', 'Venezuela, Bolivarian Republic of':'Venezuela', 
                    'Viet Nam':'Vietnam', 'The Former Yugoslav Republic of Macedonia':'Macedonia'}
    
    for unwto, gbd in location_fix.items():
        data.loc[data['location_name'] == unwto, 'location_name'] = gbd
    if more == True:
        for unwto, gbd in visitors_fix.items():
            data.loc[data['visiting_country'] == unwto, 'visiting_country'] = gbd
    return data

for country, dataset in raw.items():
        filtered_by_cp[country] = dataset.get('CP')
        proportion_datasets[country] = {x:dataset.get(x) for x in dataset.keys() 
                                        if x in ['121', '122', '111','112', '711', '712', '1911','1912']}

SyntaxError: EOL while scanning string literal (<ipython-input-85-c13dde2a58a9>, line 35)

### 4. Append filtered dictionaries, then pivot to create individual variables. Then transform to proportions.


In [21]:
def merge_filtered(df, country, sheet):
    '''Extract data from filtered dictionaries.
       Only keeps data on countries and relevant variables. Renames variables to
       match template.
    '''
    copy = DataFrame()
    merger = DataFrame()
    frames = {'total':[], 'visiting':[]}

    #Only keep countries not regions, as well as useful indicators
    df = df[(df['CODE'] < 900) | (df['CODE'].isnull())]
    df = df.drop(['CODE', '% Change 2014-2013', 'Notes', 'NOTES'], axis=1)

    #Rename columns and convert to long
    df = df.rename(columns = {'Unnamed: 2':'visiting_country'})

    #Get rid of pesky marketshare category
    df = df.iloc[:,:-1]

    df['location_name'] = '{}'.format(country)
    df = pd.melt(df, id_vars=['REGION', 'location_name', 'visiting_country'], var_name='year_id', value_name='tourists')

    #Keep total separate from visiting countries
    frames['total'].append(df[df['visiting_country'] == 'TOTAL'])
    frames['visiting'].append(df[df['visiting_country'] != 'TOTAL'])

    #Merge visiting countries for host, keeping total separate
    merger = pd.concat(frames['total'], ignore_index=True)
    copy = pd.concat(frames['visiting'], ignore_index=True)

    #Merge location specific with totals
    merger = merger.drop(['visiting_country', 'REGION'], axis=1)
    merger = merger.rename(columns = {'tourists':'total'})
    copy = pd.merge(copy, merger, how='left', on=['location_name', 'year_id'], sort=False)

    #Make sure missing observations are coded as NaN and that only countries are kept, not NaN
    for row in copy.index:
        if (type(copy.loc[row, 'tourists']) == str) | (type(copy.loc[row, 'tourists']) == unicode):
            copy.loc[row, 'tourists'] = np.nan
        if (type(copy.loc[row, 'total']) == str) | (type(copy.loc[row, 'tourists']) == unicode):
            copy.loc[row, 'total'] = np.nan

    #Get rid of countries with name NaN
    copy = copy[copy['visiting_country'] == copy['visiting_country']]
    copy['sheet'] = sheet
    
    copy = fix_locations(copy, more=True)
    return copy

proportion_datasets_clean=[]

#Merge each category group from data extracted
for key_country in proportion_datasets.keys():
    for key_sheet in proportion_datasets[key_country]:
        
        df = proportion_datasets[key_country][key_sheet]
        cleaned = merge_filtered(df, key_country, key_sheet)
        
        cleaned['tourist_proportion'] = np.log((cleaned['tourists']/(cleaned['total'])).astype(float))
        cleaned = cleaned[['location_name', 'visiting_country', 'year_id', 'tourist_proportion', 'sheet']]
        cleaned.sort_values(['location_name', 'visiting_country', 'year_id'], inplace=True)
        
        proportion_datasets_clean.append(cleaned)

combine = pd.concat(proportion_datasets_clean)
combine = combine.groupby(['location_name', 'visiting_country', 'year_id'])['tourist_proportion'].mean().reset_index()

combine.to_csv("/share/scratch/users/mgriswol/tourism_imputed/tourist_proportions.csv", index=False, encoding='utf-8')

/snfs2/HOME/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/snfs2/HOME/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/ipykernel/__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/snfs2/HOME/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/p

Traceback (most recent call last):
  File "/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/IPython/core/ultratb.py", line 376, in _fixed_getinnerframes
    lines = ulinecache.getlines(file)[start:end]
  File "/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/IPython/utils/ulinecache.py", line 37, in getlines
    return [l.decode(encoding, 'replace') for l in lines]
  File "/homes/mgriswol/.conda/envs/my_python/lib/python2.7/encodings/utf_8.py", line 16, in decode
    return codecs.utf_8_decode(input, errors, True)
KeyboardInterrupt


IndexError: string index out of range

### 5. Estimate full time series for tourist_proportions using amelia & mice regression

See R Code  
Proportion datasets were concatted together after running above cells.


In [16]:
%load_ext rpy2.ipython

In [16]:
%%R

library(data.table)
library(plyr)
source("/home/j/temp/central_comp/libraries/current/r/get_location_metadata.R")

dir <- "/share/scratch/users/mgriswol/tourism_imputed/"
locs <- get_location_metadata(location_set_id = 1)[, .(location_id, location_name)]

#Load tourist data
df <- fread(paste0(dir, "tourist_proportions.csv"))
df[, id:=paste(df$location_name, df$visiting_country, sep=",")]

#Load covariate data
covar <- fread(paste0(dir, "covariates.csv"))
covar[, cigarettes_pc:=NULL]

#Load alcohol predictions
alc <- fread(paste0(dir, "alc_gpr_2016.csv"))
alc <- join(alc, locs, by='location_id', type='left')
alc <- melt(alc, id.vars=c("location_id", "location_name", "year_id"), value.name='alc_lpc', variable.name='draw')
alc <- alc[, mean(alc_lpc, na.rm=T), by=c("location_name", "year_id")]
setnames(alc, "V1", "alc_lpc")

#Combine covariates with alcohol, merge onto tourist data.
predictors <- join(covar, alc, by=c("location_name", "year_id"), type="left")
predictors[, data:=NULL]
predictors <- predictors[!duplicated(predictors),]

final <- join(df, predictors, by=c("location_name", "year_id"), type="left")

#Make location id
locations <- data.table(location_id = seq(1, length(unique(final$location_name))), location_name = unique(final$location_name))
final <- join(final, locations, by="location_name", type="left")

#Overimpute for observed cases, parallelized on cluster
write.csv(final, paste0(dir, "final.csv"))
R_shell <- paste0(dir,"r_shell.sh")
error_path <- paste0(" -o /share/scratch/users/mgriswol/", " -e /share/scratch/users/mgriswol/")

for (loc in locations$location_id){
  
  #Build qsub
  name <- paste0("impute_case_",i)
  script <- paste0(dir,"impute.R")
  arguments <- paste(dir, i)
  
  qsub <- paste0("qsub -N ", name, 
                 " -pe multi_slot ", 2,
                 " -l mem_free=", 4,
                 error_path
                 )
  #Run qsub
  system(paste(qsub, R_shell, script, arguments))
  i <- i+1
  #print(paste(qsub, R_shell, script, arguments))
}


/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: data.table 1.9.6  For help type ?data.table or https://github.com/Rdatatable/data.table/wiki

  warnings.warn(x, RRuntimeWarning)
/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The fastest way to learn (by data.table authors): https://www.datacamp.com/courses/data-analysis-the-data-table-way

  warnings.warn(x, RRuntimeWarning)


Read 27935 rows and 1005 (of 1005) columns from 0.310 GB file in 00:00:09


In [108]:
%%R

#Loop through cases

library(data.table)
library(Amelia)

arg <- commandArgs()[-(1:3)]

dir <- "/share/scratch/users/mgriswol/tourism_imputed/"
#arg[1]
loc <- 8
#arg[2]

print(loc)
print(dir)

df <- fread(paste0(dir, "final.csv"))
df <- df[location_id==loc,]

#Check that dataset isn't already complete

if (dim(df[is.na(tourist_proportion)])[1] == 0){

    df <- df[, .(location_name, visiting_country, year_id, tourist_proportion, id, location_id)]
    df <- df[, imputed := 1]
    write.csv(df, paste0(dir, "imputed_", loc, ".csv"))
    stop("Nothing to impute")
    
}

#Check that covariates exist
for (covariate in c("ln_ldi_pc", "educ_25plus", "alc_lpc")){
  if (nrow(df[is.na(get(covariate)), ])>=0){
    df[, (covariate):=NULL]
  }
}

a <- amelia(df, m=10, 
            cs="id", 
            ts="year_id", 
            p2s = 2,
            idvars=c("location_name", "visiting_country", "location_id"), 
            polytime = 1, 
            intercs = T,
            empri = 0.01*dim(df)[1],
            parallel = "multicore")

write.amelia(a, separate = FALSE, orig.data = TRUE, file.stem = paste0(dir, "imputed_", loc), impvar = "imputed", extension = ".csv", format="csv")


[1] 8
[1] "/share/scratch/users/mgriswol/tourism_imputed/"
Error in withVisible({ : Nothing to impute


/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in withVisible({ : Nothing to impute

  warnings.warn(x, RRuntimeWarning)


In [110]:
%%R

library(data.table)
library(plyr)

folder <- "/share/scratch/users/mgriswol/tourism_imputed/"
setwd(folder)

#Read in only the files we want and append together in a datatable
files <- list.files(folder, pattern="^imputed.*csv$")
impute <- lapply(files, fread)
impute <- data.table(rbindlist(impute))

#Remove some columns and transform data
impute[, c("V1", "id", "location_id"):=NULL]
impute[, tourist_proportion:=exp(tourist_proportion)]

#Scale proportions to 1
impute[, tourist_proportion := tourist_proportion/sum(.SD[,tourist_proportion], na.rm=TRUE), 
           by=list(location_name, year_id, imputed)]

#Split data into raw data and estimates
data <- impute[imputed==0,]
data[, imputed:=NULL]

impute <- impute[imputed!=0,]

#Calculate estimates and uncertainty
impute[, `:=`(mean = mean(tourist_proportion),
             lower = quantile(tourist_proportion, 0.05),
             upper = quantile(tourist_proportion, 0.95)),
       by=.(location_name, visiting_country, year_id)]

#Collapse dataset and compare to original data
impute[, c("tourist_proportions", "imputed"):=NULL]
impute <- unique(impute)

data[, imputed:=NULL]
results <- join(data, impute, by=c("location_name", "visiting_country", "year_id"), type="full")

write.csv(results, "/share/scratch/users/mgriswol/tourism_imputed/final_tourist_proportions.csv")

In [111]:
#Combine all of the best datasets for each country pair.

from db_queries import get_location_metadata  
locations = get_location_metadata(location_set_id=1)[['location_name', 'location_id']]

#Replace frames with Amelia output
tourist_proportions = pd.read_csv("/share/scratch/users/mgriswol/tourism_imputed/final_tourist_proportions.csv")

#Format so that dataset merges correctly with later datasets
fix = fix_locations(tourist_proportions, more=True)
fix.rename(columns={'location_name':'host', 'visiting_country':'location_name'}, inplace=True)
fix = pd.merge(fix, locations, how='left', on=['location_name'])
fix = fix[['year_id', 'host', 'location_name', 'mean', 'lower', 'upper', 'location_id']]
fix.rename(columns={'host':'location_name', 'location_name':'visiting_country', 'location_id':'location_id_visitor'}, inplace=True)
fix = fix[fix.location_id_visitor==fix.location_id_visitor]

tourist_proportions = fix

### 6. Split cp by filtered proportions predictions to produce full time series for tourism for all countries


In [86]:
games = []

def merge_cp(data, country):
    '''Returns cleaned dataset for total tourism'''

    #Only keep relevant variables, rename, and transform the units.
    copy = data.iloc[[4, 5, 6, 54]]
    copy = copy.drop(['Cod.', 'Notes', 'Units'], axis=1)
    copy.iloc[0,0] = 'tourist_total'
    copy.iloc[1,0] = 'overnight_visitors'
    copy.iloc[2,0] = 'same_day_visitors'
    copy.iloc[3,0] = 'length_of_stay'
    copy.iloc[:-1,1:] = copy.iloc[:-1,1:]*1000

    #Reshape by years, then make separate columns
    copy = pd.melt(copy, id_vars=['Basic data and indicators'], var_name='year_id', value_name='data')
    copy = copy.pivot(index='year_id', columns='Basic data and indicators', values='data')
    copy['location_name'] = country
    copy['year_id'] = copy.index

    #Replace almost all missing values with next best estimates
    copy = next_best(copy)
    return copy

def next_best(data):
    '''Replaces tourist total with next best estimate if tourist total is missing'''

    if np.isnan(data['tourist_total'].values).sum() >= 19:
        data['tourist_total'] = data['overnight_visitors']
    if np.isnan(data['tourist_total'].values).sum() >= 19:
        data['tourist_total'] = data['same_day_visitors']
    return data

for country, data in filtered_by_cp.items():
    games.append(merge_cp(data, country))

#Merge on stgpr template
tourist_total = pd.concat(games, ignore_index=True)
tourist_total = fix_locations(tourist_total)
tourist_total_gpr = pd.merge(tourist_total, template, how='right', on=['location_name', 'year_id'])
tourist_total_gpr['year_id'] = tourist_total_gpr['year_id'].astype(int)

### 7. Prep tourist_total for ST-GPR

In [ ]:
def lowess(data, fraction):
    '''Calculates lowess and returns predictions'''

    x = data['year_id']
    y = data['data']

    prediction = sm.nonparametric.lowess(y, x, frac=fraction, it=10, missing='drop', return_sorted=False)

    return(prediction)

#Rename columns for gpr
tourist_total_gpr.rename(columns={'tourist_total':'data'}, inplace=True)

#Generate variance and SD using difference from lowess estimates
grouped = tourist_total_gpr.groupby('ihme_loc_id')
dames = []

for country, data in grouped:

    #Only run lowess for models with atleast 2 data points. For those with a small amount, use all of the data.
    check = data['data'].values
    check = np.isnan(check)

    if sum(~check) >= 7:
        lowess_hat = lowess(data, .6)
        data['lowess_hat'] = lowess_hat
        dames.append(data)
    if sum(~check) >= 4 and sum(~check) <= 6:
        lowess_hat = lowess(data, 1)
        data['lowess_hat'] = lowess_hat
        dames.append(data)

lowess_hat = pd.concat(dames, ignore_index=True)
lowess_hat = lowess_hat[['location_name', 'year_id', 'data', 'lowess_hat']]
lowess_hat.sort_values(['location_name', 'year_id'], inplace=True)

#Collect lowess predictions with gpr prep dataset
lowess_hat = lowess_hat[['location_name', 'year_id', 'lowess_hat']]
gpr = pd.merge(tourist_total_gpr, lowess_hat, on=['location_name', 'year_id'], how='left')

gpr['residual'] = gpr['lowess_hat'] - gpr['data']

#By country, use difference between lowess estimates and data to generate variance over a 5 year window
grouped = gpr.groupby('location_name')
frames=[]

for location, data in grouped:
    data['standard_deviation'] = pd.rolling_std(data['residual'], window=5, center=True, min_periods=1)
    frames.append(data)

gpr = pd.concat(frames, ignore_index=True)

#Only hold onto variance at points where we have data. (This happens due to the rolling window)
gpr['standard_deviation'][gpr['residual'] != gpr['residual']] = np.nan
gpr['variance'] = gpr['standard_deviation']**2

gpr['constant'] = 1

#Add missing China subnational
china = template[template['location_name']=='China']
china['ihme_loc_id'] = 'CHN_44533'
china['location_id'] = 44533
china['location_name'] = 'CHN_44533'
gpr = pd.merge(gpr, china, on=['location_id', 'year_id'], how='left')

#Add on last columns needed for gpr
gpr['nid'] = 239757
gpr['me_name'] = 'total_tourists'
gpr['sample_size'] = np.nan
gpr['sex_id'] = 3
gpr['age_group_id'] = 22
gpr['age_id'] = 22

gpr.to_csv(r'J:/WORK/05_risk/risks/drugs_alcohol/data/exp/inputs/total_tourists_pre_gpr.csv')

In [115]:
length = tourist_total[['length_of_stay', 'location_name', 'year_id']]

tourist_proportions.to_csv("/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/inputs/tourist_proportions.csv", index=False)
length.to_csv("/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/inputs/length_of_stay.csv", index=False)


In [137]:
%%R

df[is.na(length_of_stay), unique(location_name)]

  [1] "Canada"                                          
  [2] "Turkmenistan"                                    
  [3] "Cote d'Ivoire"                                   
  [4] "Iran"                                            
  [5] "Cambodia"                                        
  [6] "Ethiopia"                                        
  [7] "Aruba"                                           
  [8] "Swaziland"                                       
  [9] "Cameroon"                                        
 [10] "Burkina Faso"                                    
 [11] "Bahrain"                                         
 [12] "Saudi Arabia"                                    
 [13] "Bolivia"                                         
 [14] "American Samoa"                                  
 [15] "Northern Mariana Islands"                        
 [16] "Slovenia"                                        
 [17] "Guatemala"                                       
 [18] "Guinea"                 

### 8. Bring in GPR results on LPC and use this, along with transformed tourism data, to create tourism adjustments


The following equations describe the adjustment:

$$ \text{Adjusted LPC}_h = \text{Observed LPC}_h + \text{LPC}_\textit{a} - \text{LPC}_v $$

$$ \text{LPC}_a = \frac{\sum_v (\text{Unadjusted LPC}_h * \text{Tourist pop}_\text{a,v})}{\text{Population}_h}$$

$$ \text{LPC}_v =  \frac{\sum_v (\text{Unadjusted LPC}_v * \text{Tourist pop}_\text{h,v})}{\text{Population}_h}$$

$$ \text{Tourist pop}_\text{i,v} = \text{Proportion of tourists}_{i,v} * \text{Tourist pop}_v * \frac{\text{Trip duration}_{i,v}}{365} \; \; \text{for }i={a,h}$$  

where h is a hosting country, a is citizens from a hosting country traveling abroad, and v is a visiting country.

In [6]:
import os

alc_lpc = pd.read_csv("/snfs1/WORK/05_risk/risks/drugs_alcohol/data/exp/stgpr/alc_gpr_2017.csv")

locs = alc_lpc.location_id.unique()
locs = [533, 361, 354]

for loc in locs:
    
    qsub = "qsub -P proj_custom_models -l mem_free=8 -pe multi_slot 4 -N net_tourism_" + str(loc) +\
            " -o /share/temp/sgeoutput/mgriswol -e /share/temp/sgeoutput/mgriswol " + \
            "/share/gbd/WORK/05_risk/02_models/02_results/drugs_alcohol/code/r_shell.sh " + \
            "/share/gbd/WORK/05_risk/02_models/02_results/drugs_alcohol/code/net_tourism.R " + str(loc)

    os.system(qsub)

In [1]:
%%writefile /share/gbd/WORK/05_risk/02_models/02_results/drugs_alcohol/code/net_tourism.R

rm(list=ls())
arg <- commandArgs()[-(1:3)]

library(plyr)
library(data.table)
library(dplyr)

#Read in needed inputs
loc <- as.numeric(arg[1])

main_dir   <- "/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/"
input_dir  <- paste0(main_dir, "inputs/")
output_dir <- paste0(main_dir, "outputs/")
final_dir  <- "/share/scratch/users/mgriswol/net_adjustment/"

template            <- fread(paste0(input_dir, "tourism_template.csv"))
tourist_proportions <- fread(paste0(input_dir, "tourist_proportions.csv"))
tourist_totals      <- fread(paste0(main_dir, "stgpr/tourism_gpr_2016.csv"))
trip_duration       <- fread(paste0(input_dir, "length_of_stay.csv"))
alc_lpc             <- fread(paste0(main_dir, "stgpr/alc_gpr_2017.csv"))

loc_name <- template[location_id==loc, unique(location_name)]

#Get alcohol lpc and tourist totals from gpr. Reshape long and hold onto needed columns, discarding rest. Add on ids.
alc_lpc <-  join(alc_lpc, template, by=c("location_id", "year_id"), type="left") %>%
            melt(., id.vars=c("location_id", "location_name", "year_id", "pop_scaled"), 
                measure.vars=paste0("draw_", 0:999), value.name='alc_lpc', variable.name="draw") %>%
            .[!is.na(location_name)]

tourist_totals <- melt(tourist_totals, id.vars = c("location_id", "year_id"), measure.vars = paste0("draw_", 0:999),
                        value.name='total_tourists', variable.name='draw')

tourist_proportions <- setnames(tourist_proportions, "mean", "tourist_proportion") %>%
                        .[(location_name == loc_name | location_id_visitor == loc),] %>%
                        unique %>%
                        .[!is.na(location_id_visitor)]

trip_duration <- join(trip_duration, template, by=c("location_name", "year_id"), type = "left") %>%
                    .[, .(location_name, location_id, year_id, length_of_stay)] %>%
                    .[!is.na(location_id)]

population <- template[location_id == loc, .(location_id, year_id, pop_scaled)]

#Separate proportions into consumption that locals consume while abroad and consumption tourists consume while visiting locally
abroad   <- tourist_proportions[location_id_visitor == loc, ] %>%
            join(., template, by=c("location_name", "year_id"), type = "left") %>%
            .[, .(location_name, location_id, year_id, visiting_country, location_id_visitor, tourist_proportion)] %>%
            .[, year_id := as.numeric(year_id)] %>%
            .[!is.na(location_id)]

domestic <- tourist_proportions[location_name == loc_name, ] %>%
            join(., template, by=c("location_name", "year_id"), type = "left") %>%
            .[, .(location_name, location_id, year_id, visiting_country, location_id_visitor, tourist_proportion)] %>%
            .[, year_id := as.numeric(year_id)]

#Calculate average length of stay and fill in for missing observations. Convert to fraction of year
average_duration <- trip_duration[, mean(length_of_stay, na.rm=TRUE)]
trip_duration    <- trip_duration[is.na(length_of_stay), length_of_stay := average_duration]

#Construct additive and subtractive measure of tourists for given location:
tourist_consumption <- function(h, y, v, tourist_prop, tourist_total, lpc, duration){
    
    #For a given host country, visiting country, and year, calculate 1000 draws of tourist consumption
    
    tourist_prop   <- tourist_prop[J(h, v, y), tourist_proportion]
    tourist_total  <- tourist_total[J(h, y), total_tourists]
    lpc            <- lpc[J(v,y), alc_lpc]
    duration       <- trip_duration[J(v, y), length_of_stay]

    consumption <- lpc * tourist_prop * tourist_total * (duration/365)
    tourist_consumption <- data.table(host = h, visitor = v, year_id = y, 
                                      draw = paste0("draw_", 0:999), tourist_consumption = consumption)
    
    return(tourist_consumption)
}

setkey(abroad, location_id, location_id_visitor, year_id)
setkey(domestic, location_id, location_id_visitor, year_id)
setkey(tourist_totals, location_id, year_id)
setkey(alc_lpc, location_id, year_id)
setkey(trip_duration, location_id, year_id)

args <- unique(abroad[, .(location_id, year_id)])
tourist_consumption_abroad <- mdply(cbind(h = as.numeric(args$location_id), y = as.numeric(args$year_id)), 
                                    tourist_consumption, v = loc, tourist_prop = abroad, tourist_total = tourist_totals, 
                                    duration = trip_duration, lpc = alc_lpc) %>% 
                              as.data.table %>%
                              .[, sum(.SD$tourist_consumption, na.rm=T), by=c('year_id', 'draw')] %>%
                              setnames(., "V1", "tourist_consumption") %>%
                              join(., population, by="year_id", type="left") %>%
                              .[, tourist_consumption := tourist_consumption/pop_scaled]

args <- unique(domestic[, .(location_id_visitor, year_id)])
tourist_consumption_domestic <-  mdply(cbind(v = as.numeric(args$location_id), y = as.numeric(args$year_id)), 
                                    tourist_consumption, h = loc, tourist_prop = domestic, tourist_total = tourist_totals, 
                                    duration = trip_duration, lpc = alc_lpc) %>% 
                                 as.data.table %>%
                                 .[, sum(.SD$tourist_consumption, na.rm=T), by=c('year_id', 'draw')] %>%
                                 setnames(., "V1", "tourist_consumption") %>%
                                 join(., population, by="year_id", type="left") %>%
                                 .[, tourist_consumption := tourist_consumption/pop_scaled]

net_tourism <- tourist_consumption_abroad$tourist_consumption - tourist_consumption_domestic$tourist_consumption

net <- expand.grid(location_id = loc, location_name = loc_name, year_id = unique(tourist_consumption_domestic$year_id),
                   draw = unique(tourist_consumption_domestic$draw)) %>% cbind(., net_tourism) %>% data.table

write.csv(net, paste0(final_dir, "net_adjustment_", loc, ".csv"), row.names=F)

Overwriting /share/gbd/WORK/05_risk/02_models/02_results/drugs_alcohol/code/net_tourism.R


In [7]:
#Grab all of the collapsed datasets and append them together. 

os.chdir("/share/scratch/users/mgriswol/net_adjustment/")
tourism_adjustments = []

for files in os.listdir(os.getcwd()):
    if files.startswith("net_adjustment_"):
        tourism_adjustments.append(pd.read_csv(files))

tourism_adjustments = pd.concat(tourism_adjustments)
tourism_adjustments.to_csv('{}/tourism_adjustments.csv'.format(outgoing), index=False)

In [11]:
#%%Backcast average estimates of last three years to the years 1980-1995 to match the covariate, along with forecasting to 2017
#(Better method would be to account for changes in host country populations in addition to using this average,
#i.e. only divide by pop_scaled after having back and forecasted values)
#Runs in parallel

from multiprocessing import Pool

tourism_adjustments = pd.read_csv('{}/tourism_adjustments.csv'.format(outgoing))
backcast = tourism_adjustments.groupby(['location_id', 'draw', 'location_name'])
                                        
def caster(params):
    
    keys = params[0]
    df = params[1]
    
    addon = pd.DataFrame({'year_id':years})
    addon['location_id'] = keys[0]
    addon['draw'] = keys[1]
    addon['location_name'] = keys[2]
    addon['net_tourism'] = df[df.year_id.isin([1995,1996,1997])]['net_tourism'].mean()
    addon.loc[(addon.year_id >= 2015), 'net_tourism'] = df[df.year_id.isin([2012,2013,2014])]['net_tourism'].mean()
    
    return(pd.concat([df, addon], ignore_index=True))

#Bind parameters together (pool requires passing single arguments) based on keys of groupby and constituent dataframes
params = []
for keys, df in backcast:
    
    params.append((keys, df))

p = Pool(40)
final_adjustment = p.map(caster, params)
final_adjustment = pd.concat(final_adjustment, ignore_index=True)
final_adjustment = final_adjustment.loc[final_adjustment.net_tourism == final_adjustment.net_tourism]

final_adjustment.to_csv('{}/final_tourist_adjustment.csv'.format(outgoing), index=False)

In [12]:
final_adjustment.to_csv('{}/final_tourist_adjustment.csv'.format(outgoing), index=False)

In [13]:
final_adjustment[final_adjustmentadjustment.year_id==2017]

,draw,location_id,location_name,net_tourism,year_id
36,draw_0,6,China,-0.000217,2017
73,draw_1,6,China,-0.000223,2017
110,draw_10,6,China,-0.000361,2017
147,draw_100,6,China,-0.000284,2017
184,draw_101,6,China,-0.000406,2017
221,draw_102,6,China,-0.000330,2017
258,draw_103,6,China,-0.000279,2017
295,draw_104,6,China,-0.000246,2017
332,draw_105,6,China,-0.000383,2017
369,draw_106,6,China,-0.000278,2017


In [169]:
outgoing

'/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/outputs'

### 9. Combine with alc_lpc gpr results and export

In [14]:
#Load in alc lpc estimates, hold onto relevant columns and rows, melt to long
alc_lpc = pd.read_csv("/snfs1/WORK/05_risk/risks/drugs_alcohol/data/exp/stgpr/alc_gpr_2017.csv")
alc_lpc = alc_lpc[['year_id','location_id']+draws]
alc_lpc = alc_lpc[alc_lpc['year_id'] >= 1990]
alc_lpc = pd.melt(alc_lpc, id_vars=['year_id', 'location_id'], var_name="draw", value_name="alc_lpc")

#Merge with adjustments, add in net. Only use adjustment if above 0 after applying adjustment (could not be the case due to assumptions above & poor data in alc lpc FAO)
adj_alc = pd.merge(alc_lpc, final_adjustment, how="left", on=["location_id", "year_id", "draw"])
adj_alc['net_tourism'].fillna(0, inplace=True)
adj_alc['alc_lpc_new'] = adj_alc["alc_lpc"] + adj_alc["net_tourism"]
adj_alc.loc[(adj_alc.alc_lpc_new <= 0), 'alc_lpc_new'] = adj_alc.loc[(adj_alc.alc_lpc_new <= 0), 'alc_lpc']

adj_alc['alc_lpc'] = adj_alc['alc_lpc_new']

#Clean up dataframe and send off
adj_alc = adj_alc[['year_id', 'location_id', 'draw', 'alc_lpc', 'net_tourism']]
adj_alc.to_csv("{}/adj_alc_lpc_2017.csv".format(outgoing), index=False)


In [18]:
%%R

#Collapse tourism estimates for vetting.

library(data.table)
library(dplyr)
library(plyr)
source("/home/j/temp/central_comp/libraries/current/r/get_location_metadata.R")

locs <- get_location_metadata(location_set_id = 1) %>% .[ ,.(location_id, level, location_name, region_name, super_region_name)]

df <- fread('/home/j/WORK/05_risk/risks/drugs_alcohol/data/exp/outputs/final_tourist_adjustment.csv')
df <- df[year_id==2015]

df <- df[, `:=`(mean_val = mean(.SD$net_tourism),
                lower_val = quantile(.SD$net_tourism, 0.025),
                upper_val = quantile(.SD$net_tourism, 0.975)),
               by=c('location_id', 'year_id')] %>%
        .[, .(location_id, year_id, mean_val, lower_val, upper_val)] %>%
        unique %>%
        join(., locs, by='location_id') %>%
        .[level==3,]

write.csv(df, '/snfs2/HOME/mgriswol/tourism_2017.csv', row.names=F)

KeyboardInterrupt: 

/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ------------------------------------------------------------------------------

  warnings.warn(x, RRuntimeWarning)
/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

  warnings.warn(x, RRuntimeWarning)
/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘plyr’


  warnings.warn(x, RRuntimeWarning)
/homes/mgriswol/.conda/envs/my_python/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summariz

Read 6808000 rows and 5 (of 5) columns from 0.280 GB file in 00:00:10
